In [1]:
import pandas as pd




In [2]:
df=pd.read_csv("result.csv")

In [3]:
# 中止になった試合が含まれているので除外する
df_cleaned = df.dropna(subset=['Batting_first_runs'])
df_cleaned

,year,Batting_first_inning,Batting_first_runs,Batting_second_inning,Batting_second_runs
0,1915,7.0,1.0,8.0,2.0
1,1916,4.0,2.0,3.0,6.0
2,1917,14.0,1.0,NaN,0.0
4,1919,6.0,4.0,2.0,7.0
5,1920,NaN,0.0,3.0,17.0
...,...,...,...,...,...
102,2021,1.0,9.0,2.0,2.0
103,2022,6.0,1.0,4.0,8.0
104,2023,1.0,8.0,2.0,2.0
105,2024,10.0,2.0,10.0,1.0


In [4]:
# 先制したチームが先攻チームか後攻チームかを判定する
# 同じイニングに得点を挙げている場合は先攻チームが先制したチームと判定
scored_first=[]
for index in df_cleaned.index:
    first_inning_score = df_cleaned['Batting_first_inning'][index]
    second_inning_score = df_cleaned['Batting_second_inning'][index]

    if pd.isna(first_inning_score):
        scored_first.append('Batting_second')
    elif pd.isna(second_inning_score):
        scored_first.append('Batting_first')
    else:
        difference_value = first_inning_score - second_inning_score
        if difference_value == 0 or difference_value < 0:
            scored_first.append('Batting_first')
        else:
            scored_first.append('Batting_second')

In [5]:
df_cleaned.insert(loc=5, column='scored_first', value=scored_first)

In [6]:
# 勝利したチームが先攻チームか後攻チームかを判定する
victory=[]
difference_value=df_cleaned['Batting_first_runs']-df_cleaned['Batting_second_runs']

for value in difference_value:
    if pd.isna(value):
        print('XXX')
        # 差がNaNの場合は、スキップするか、特定の値を割り当てるかなどを検討
        # 例としてここではスキップします
        pass
    elif value > 0:
        victory.append('Batting_first')
    elif value < 0:
        victory.append('Batting_second')
    else: # value == 0 の場合
        # 引き分けの場合の処理を検討
        # 例としてここでは'Draw'とします
        victory.append('Draw')


In [10]:
# Drawがないことを確認
DRAW_NUM=victory.count('Draw')
if DRAW_NUM==0:
    print('引き分けなし')
else:
    print('引き分け', DRAW_NUM, '回あり')

引き分けなし


In [8]:
df_cleaned.insert(loc=6, column='victory team', value=victory)
victory_first_runs = df_cleaned['scored_first'] == df_cleaned['victory team']

df_cleaned.insert(loc=7, column='先制したチームが勝つ', value=victory_first_runs)
df_cleaned

,year,Batting_first_inning,Batting_first_runs,Batting_second_inning,Batting_second_runs,scored_first,victory team,先制したチームが勝つ
0,1915,7.0,1.0,8.0,2.0,Batting_first,Batting_second,False
1,1916,4.0,2.0,3.0,6.0,Batting_second,Batting_second,True
2,1917,14.0,1.0,NaN,0.0,Batting_first,Batting_first,True
4,1919,6.0,4.0,2.0,7.0,Batting_second,Batting_second,True
5,1920,NaN,0.0,3.0,17.0,Batting_second,Batting_second,True
...,...,...,...,...,...,...,...,...
102,2021,1.0,9.0,2.0,2.0,Batting_first,Batting_first,True
103,2022,6.0,1.0,4.0,8.0,Batting_second,Batting_second,True
104,2023,1.0,8.0,2.0,2.0,Batting_first,Batting_first,True
105,2024,10.0,2.0,10.0,1.0,Batting_first,Batting_first,True


In [ ]:
import statsmodels.api as sm

COUNT=df_cleaned['先制したチームが勝つ'].sum()
NOBS=len(df_cleaned['先制したチームが勝つ'])
p=0.5

stat, p_value = sm.stats.proportions_ztest(count=COUNT, nobs=NOBS, value=p, alternative='larger')
print(f"Z値: {stat:.4f}")
print(f"P値: {p_value:.4f}")

Z値: 4.5019
P値: 0.0000
